# How to connect to Azure OpenAI service via OpenAI Python SDK

First import the OpenAI Python SDK and load your API key from the environment.

Go to your Azure OpenAI resource and copy the API key and endpoint.

https://oai.azure.com/portal/


In [69]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv, dotenv_values
from pprint import pprint

#load_dotenv()


config = dotenv_values(".env")


azure_oai_key = config["AZURE_OPENAI_API_KEY"]
api_base_url = config["AZURE_OPENAI_ENDPOINT"]

deployment_name = config["AZURE_OPENAI_DEPLOYMENT_NAME"]
api_version = config["AZURE_OPENAI_API_VERSION"]

print(
    f"Azure OpenAI api key={azure_oai_key[0:3]}...{azure_oai_key[-5:]}"
)
print(
    f"Azure OpenAI api endpoint={api_base_url[0:17]}..."
)
print(f"Azure OpenAI deployment name={deployment_name[0:7]}...")
print(f"Azure OpenAI api version={api_version}")

Azure OpenAI api key=9b2...d67b3
Azure OpenAI api endpoint=https://openai-pr...
Azure OpenAI deployment name=gpt-35T...
Azure OpenAI api version=2023-07-01-preview


In [70]:
aoai_client = AzureOpenAI(
    api_key=azure_oai_key,
    azure_endpoint=api_base_url,
    api_version=api_version,
)

In [71]:

models = aoai_client.models.list().data
models = [m for m in models if "gpt" in m.id.lower()]

# sort models by model id   
models = sorted(models, key=lambda x: x.id)

# print models which contains gpt in the name
pprint(models)



[Model(id='gpt-35-turbo', created=None, object='model', owned_by=None, capabilities={'fine_tune': False, 'inference': True, 'completion': True, 'chat_completion': True, 'embeddings': False}, lifecycle_status='generally-available', deprecation={'inference': 1720137600}, status='succeeded', created_at=1678320000, updated_at=1688601600),
 Model(id='gpt-35-turbo-0301', created=None, object='model', owned_by=None, capabilities={'fine_tune': False, 'inference': True, 'completion': True, 'chat_completion': True, 'embeddings': False}, lifecycle_status='generally-available', deprecation={'inference': 1720137600}, status='succeeded', created_at=1678320000, updated_at=1688601600),
 Model(id='gpt-35-turbo-0613', created=None, object='model', owned_by=None, capabilities={'fine_tune': False, 'inference': True, 'completion': False, 'chat_completion': True, 'embeddings': False}, lifecycle_status='generally-available', deprecation={'fine_tune': 1718236800, 'inference': 1718236800}, status='succeeded', 

In [72]:
response = aoai_client.chat.completions.create(
    model=deployment_name,
    messages=[
        {
            "role": "system",
            "content": "You are teacher and help studends learn phisics, geography and chemistry.",
        },
        {
            "role": "user",
            "content": "Compose a poem that describes the soloar plantes",
        },
    ],
)

pprint(response.choices[0].message.content)

('In a realm beyond celestial veil,\n'
 "Where the universe's wonders prevail,\n"
 'Lies a collection of heavenly delights,\n'
 'Planets that dance in cosmic nights.\n'
 '\n'
 'First, the fiery Mercury burns brightly,\n'
 'A scorched ball of volcanic activity,\n'
 'Close to the sun, it races with haste,\n'
 'A tiny world, truly a marvel in space.\n'
 '\n'
 'Next, Venus, shrouded in a cloak of mist,\n'
 'With a toxic atmosphere, it does persist,\n'
 'A sister to Earth, yet a world so distinct,\n'
 'Mysteries and beauty intertwined, succinct.\n'
 '\n'
 'Then arrives our home, the beloved Earth,\n'
 'A flourishing oasis, of infinite worth,\n'
 'With oceans and continents, a wondrous sight,\n'
 'A cradle for life, bathed in sunlight.\n'
 '\n'
 'Mars, the red planet, draped in rusted hue,\n'
 'Perseverance unfading, its essence true,\n'
 'A barren landscape, but secrets it holds,\n'
 'A realm of exploration, as science unfolds.\n'
 '\n'
 'Jupiter, a behemoth, king of them all,\n'
 'A coloss

## Create a request to the OpenAI via promptsail proxy

Run the docker and go to PromptSail UI http://promptsail.local/


At the application start we created two test projects with OpenAI API mappings. We will use project1 for this example.

* [project1](http://promptsail.local/ui/project/project1) -> https://api.openai.com/v1

You can create your own project if you want.


In [62]:
import httpx

ps_api_base = "http://promptsail.local:8000/project2"


ps_client = AzureOpenAI(
    api_key=azure_oai_key,
    azure_endpoint=ps_api_base,
    api_version=api_version,
)


response = ps_client.chat.completions.create(
    model=deployment_name,
    messages=[
        {
            "role": "system",
            "content": "You are teacher and help studends learn phisics, geography and chemistry.",
        },
        {
            "role": "user",
            "content": "Name the plantets in solar system.",
        },
    ],
)

pprint(response.choices[0].message.content)



InternalServerError: Internal Server Error